## Постановка задачи
У нас появился запрос из отдела продаж и маркетинга. Как вы знаете «МегаФон» предлагает обширный набор различных услуг своим абонентам. При этом разным пользователям интересны разные услуги. Поэтому необходимо построить  алгоритм, который **для каждой пары пользователь-услуга определит вероятность подключения услуги**.

### Исходные данные

#### наборы данных
- **features.csv** (22Г): id, buy_time, <feature_list>
нормализованный анонимизированный набор признаков, характеризующий профиль потребления абонента. Эти данные привязаны к определенному времени, поскольку профиль абонента может меняться с течением времени.
- **data_train.csv** (27М): id, vas_id, buy_time, target
информация об отклике абонентов на предложение подключения одной из услуг. Каждому пользователю может быть сделано несколько предложений в разное время, каждое из которых он может или принять, или отклонить.
- **data_test.csv**: id, vas_id, buy_time
тестовый набор

#### переменные
   - **target** - целевая переменная, где 1 означает подключение услуги, 0 - абонент не подключил услугу соответственно.
   - **buy_time** - время покупки, представлено в формате timestamp, для работы с этим столбцом понадобится функция datetime.fromtimestamp из модуля datetime.
   - **id** - идентификатор абонента
   - **vas_id** - подключаемая услуга




### Анализ данных

1. считаем data_train и data_test и проверим на разнообразие услуг и абонентов
2. Восстановим наборы абонентов и услуг
3. обработаем features.csv отфильруем только то что есть в data_train и data_test!

![Схема](data_schema.png)

In [60]:
import pandas as pd
from matplotlib import pyplot as plt

Считаем исходные данные

In [16]:
data_train = pd.read_csv('data_train.csv')
data_train.head(3)

,Unnamed: 0,id,vas_id,buy_time,target
0,0,540968,8.0,1537131600,0.0
1,1,1454121,4.0,1531688400,0.0
2,2,2458816,1.0,1534107600,0.0


In [18]:
data_test = pd.read_csv('data_test.csv')
data_test.head(3)

,Unnamed: 0,id,vas_id,buy_time
0,0,3130519,2.0,1548018000
1,1,2000860,4.0,1548018000
2,2,1099444,2.0,1546808400


Объединим наборы для восстановления id абонентов

In [26]:
data_all = pd.concat([data_train.drop(columns='target'), data_test])
data_all.head(3)

,Unnamed: 0,id,vas_id,buy_time
0,0,540968,8.0,1537131600
1,1,1454121,4.0,1531688400
2,2,2458816,1.0,1534107600


In [27]:
data_all.shape[0], data_all.id.nunique(), data_all.vas_id.nunique()

(902884, 872577, 8)

> нам необходимо отфильтровать features.csv по 872 тыс абонентов

#### Эксперимент над частью профиля

Создадим для экспериментов срез файла features.csv - первые 5 тыс. записей

In [38]:
num_lines = 5000
with open("features_lite.csv", "x") as lite:
    with open("features.csv") as f:
        for line in f:
            lite.write(line)
            num_lines -= 1
            if num_lines % 100 == 0:
                print(num_lines, line[0:20])
            if num_lines <= 0:
                break

4900 98	2402038	154559880
4800 198	2813843	15462036
4700 298	3554903	15328980
4600 398	3661489	15419700
4500 498	3748859	15310836
4400 598	3863771	15468084
4300 698	3963168	15322932
4200 798	4047744	15377364
4100 898	4129298	15310836
4000 998	4201230	15322932
3900 1098	4270438	1538946
3800 1198	4344118	1535922
3700 1298	50258	154257480
3600 1398	133220	15389460
3500 1498	216934	15310836
3400 1598	293264	15377364
3300 1698	366781	15455988
3200 1798	439178	15365268
3100 1898	568256	15425748
3000 1998	879298	15310836
2900 2098	923040	15449940
2800 2198	942906	15449940
2700 2298	961432	15462036
2600 2398	981936	15377364
2500 2498	1004744	1541365
2400 2598	1040382	1532293
2300 2698	1071139	1545598
2200 2798	1089994	1533502
2100 2898	1111412	1535317
2000 2998	1128739	1544994
1900 3098	1146979	1534107
1800 3198	1165797	1546203
1700 3298	1186322	1536526
1600 3398	1241864	1544389
1500 3498	1257553	1535317
1400 3598	1278188	1544389
1300 3698	1297428	1543784
1200 3798	1317219	1542574
1100 3898	13

In [40]:
features = pd.read_csv('features_lite.csv', sep="\t")
features.head(3)

,Unnamed: 0,id,buy_time,0,1,2,3,4,5,6,...,243,244,245,246,247,248,249,250,251,252
0,0,2013026,1531688400,18.910029,46.980888,4.969214,-1.386798,3.791754,-14.01179,-16.08618,...,-977.373846,-613.770792,-25.996269,-37.630448,-301.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
1,1,2014722,1539550800,36.690029,152.400888,448.069214,563.833202,463.841754,568.99821,-16.08618,...,-891.373846,-544.770792,-20.996269,48.369552,80.252276,-13.832889,-0.694428,-1.175933,-0.45614,0.0
2,2,2015199,1545598800,-67.019971,157.050888,-63.180786,178.103202,-68.598246,156.99821,3.51382,...,-977.373846,-613.770792,-12.996269,-37.630448,10829.252276,-25.832889,-0.694428,-12.175933,-0.45614,0.0


In [42]:
features.shape

(4999, 256)

##### Фильтрация по клиентам
Проведем фильтрацию features только по клиентам  data_all

In [55]:
merge = features.id.isin(data_all.id)
features1 = features[merge]
features1.head(5)

,Unnamed: 0,id,buy_time,0,1,2,3,4,5,6,...,243,244,245,246,247,248,249,250,251,252
13,13,2046132,1534712400,300.820029,1599.480888,286.879214,1585.013202,281.461754,1563.90821,-16.08618,...,-977.373846,-613.770792,-25.996269,-35.630448,-295.747724,-17.832889,-0.694428,-4.175933,-0.45614,0.0
16,16,2050810,1540760400,-86.209971,91.820888,-84.480786,110.333202,-89.898246,89.22821,-16.08618,...,-977.373846,-613.770792,-23.996269,190.369552,-286.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
19,19,2070757,1540760400,-96.799971,-408.179112,-110.740786,-460.786798,-114.038246,-479.77179,-16.08618,...,-925.373846,-561.770792,-21.996269,-37.630448,-151.747724,-24.832889,0.305572,-12.175933,-0.45614,1.0
20,20,2071522,1544994000,-94.939971,-363.699112,-108.880786,-411.226798,-114.298246,-432.33179,-16.08618,...,-977.373846,-613.770792,-25.996269,-37.630448,-306.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
22,22,2075318,1533502800,-75.639971,669.690888,-89.580786,732.343202,-94.998246,736.65821,-16.08618,...,-501.373846,-242.770792,-25.996269,-37.630448,-167.747724,-14.832889,2.305572,-4.175933,-0.45614,0.0


In [54]:
features1.id.value_counts().sum(), features.id.count()

(946, 4999)

> по предварительной выборке можно сократить набор features в 5 раз

##### Пропуски

In [57]:
print("ID уникален? ", features1.id.is_unique)
print("Есть ли дубли в строках?", features1.duplicated().sum())
print("Сколько процент признаков могут принимать null-значениями? %d%%" % float(
    (features1.isnull().sum() > 0).sum() / features1.shape[1] * 100))

ID уникален?  True
Есть ли дубли в строках? 0
Сколько процент признаков могут принимать null-значениями? 0%


##### Анализ типа переменных
Просмотрим визуально признаки  features

In [56]:
features1.head(10).T

,13,16,19,20,22,27,33,34,39,43
Unnamed: 0,1.300000e+01,1.600000e+01,1.900000e+01,2.000000e+01,2.200000e+01,2.700000e+01,3.300000e+01,3.400000e+01,3.900000e+01,4.300000e+01
id,2.046132e+06,2.050810e+06,2.070757e+06,2.071522e+06,2.075318e+06,2.085648e+06,2.102126e+06,2.104377e+06,2.133447e+06,2.212313e+06
buy_time,1.534712e+09,1.540760e+09,1.540760e+09,1.544994e+09,1.533503e+09,1.539551e+09,1.538341e+09,1.532293e+09,1.535317e+09,1.537736e+09
0,3.008200e+02,-8.620997e+01,-9.679997e+01,-9.493997e+01,-7.563997e+01,6.844003e+01,1.998100e+02,-9.679997e+01,3.544800e+02,-2.899997e+01
1,1.599481e+03,9.182089e+01,-4.081791e+02,-3.636991e+02,6.696909e+02,2.604309e+02,1.850109e+02,-4.081791e+02,5.370089e+01,-6.071911e+01
...,...,...,...,...,...,...,...,...,...,...
248,-1.783289e+01,-2.583289e+01,-2.483289e+01,-2.583289e+01,-1.483289e+01,-6.832889e+00,-8.832889e+00,3.216711e+01,-8.328886e-01,-2.483289e+01
249,-6.944285e-01,-6.944285e-01,3.055715e-01,-6.944285e-01,2.305572e+00,-6.944285e-01,-6.944285e-01,-6.944285e-01,-6.944285e-01,-6.944285e-01
250,-4.175933e+00,-1.217593e+01,-1.217593e+01,-1.217593e+01,-4.175933e+00,-5.175933e+00,4.824067e+00,4.582407e+01,1.282407e+01,-1.217593e+01
251,-4.561399e-01,-4.561399e-01,-4.561399e-01,-4.561399e-01,-4.561399e-01,-4.561399e-01,-4.561399e-01,-4.561399e-01,-4.561399e-01,5.438601e-01


Часть признаков имеют категориальный/бинарный характер: 6, 8, 9, 10, 11, 12... Поэтому можно провести более глубокий анализ

In [109]:
class FeaturesInspect:
    dataframe = None
    feat_nunique = None
    feats = dict()

    def __init__(self, dataframe, num_cat=10):
        self.dataframe = dataframe
        self.feat_nunique = self.dataframe.apply(lambda x: x.nunique(dropna=False))
        self.num_cat = num_cat

    def show_nunique(self):
        plt.title("Распределение уникальных значений признаков")
        self.feat_nunique.hist(bins=100, figsize=(10, 5))

    def collect(self):
        self.feats['all'] = set(self.feat_nunique.index.tolist())
        self._collect_const()
        self._collect_numeric()
        self.feats['other'] = self.feats['all'] - (self.feats['numeric'] | self.feats['const'])
        self._collect_binary()
        self._collect_categorical()
        self.feats['extra'] = self.feats['categorical']
        self.feats['ok'] = self.feats['binary'] | self.feats['categorical'] | self.feats['numeric']

    def print(self):
        print('Всего уникальных признаков :', len(self.feats['all']))
        print('...константные  признаки :', len(self.feats['const']))
        print('...вещественные признаки :', len(self.feats['numeric']))
        self.feats['other'] = self.feats['all'] - (self.feats['numeric'] | self.feats['const'])
        print('...другие признаки :', len(self.feats['other']))
        print('...бинарные признаки :', len(self.feats['binary']))
        print('...категориальные признаки :', len(self.feats['categorical']))
        self.feats['extra'] = self.feats['categorical']
        self.feats['ok'] = self.feats['binary'] | self.feats['categorical'] | self.feats['numeric']

    def _collect_const(self):
        self.feats['const'] = set(self.feat_nunique[self.feat_nunique == 1].index.tolist())
        return len(self.feats['const'])

    def _collect_numeric(self):
        f_numeric = (self.dataframe.fillna(0).astype(int).sum() - self.dataframe.fillna(0).sum()).abs()
        self.feats['numeric'] = set(f_numeric[f_numeric > 0].index.tolist())
        return len(self.feats['numeric'])

    def _collect_categorical(self):
        self.feats['categorical'] = set(
            self.feat_nunique.loc[self.feats['other']][
                self.feat_nunique.loc[self.feats['other']] <= self.num_cat].index.tolist())
        return len(self.feats['categorical'])

    def _collect_binary(self):
        f_other = self.feats['other']
        self.feats['binary'] = set(self.dataframe.loc[:, f_other].columns[(
                (self.dataframe.loc[:, f_other].max() == 1) &
                (self.dataframe.loc[:, f_other].min() == 0) &
                (self.dataframe.loc[:, f_other].isnull().sum() == 0))])
        return len(self.feats['binary'])


In [116]:
feat_inspector = FeaturesInspect(features, num_cat=10)
feat_inspector.feat_nunique.shape

(256,)

In [117]:
#feat_inspector.show_nunique()

In [118]:
feat_inspector.collect()
feat_inspector.print()

Всего уникальных признаков : 256
...константные  признаки : 13
...вещественные признаки : 247
...другие признаки : 4
...бинарные признаки : 1
...категориальные признаки : 1
